In [1]:
#meta 2/21/2010
# same as beginner.ipynb + me looking into datastructs

#meta 8/15/2022 Org

#infra: Wlaptop 
#env anya_tf2
#py 3.7.4 
#tf 2.6.3
#numpy 1.21.0, mpl 3.1.3 (not used here: pandas 1.0.3, sns 0.10.1)

#history
#8/15/2022 ORG
#      Image classifier with TF2; fashion mnist

##### Copyright 2019 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install the TensorFlow 2 package. Import TensorFlow into your program:

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [18]:
#python version $my
import sys
print(sys.version)
print(tf.__version__)

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
2.6.3


## 0. Load Data
Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). 

myNote: The MNIST database of handwritten digits has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.
It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.


In [4]:
mnist = tf.keras.datasets.mnist # class module
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
#myPreview data
x_train.__class__ # class ndarray
print("Shape: ", x_train.shape) #(60000, 28, 28)
print("Shape of one record: ", x_train[0].shape)
x_train[24][12]

Shape:  (60000, 28, 28)
Shape of one record:  (28, 28)


array([  0,   0,   0,   0,   0,   0,  41, 216, 254, 254, 239, 153,  37,
         4,  32, 254, 254,  85,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=uint8)

Convert the samples from integers to floating-point numbers:

In [6]:

x_train, x_test = x_train / 255.0, x_test / 255.0

## 1. Model
Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [8]:
predictions = model(x_train[:1]).numpy()
print("Shape: ", predictions.shape)
predictions

Shape:  (1, 10)


array([[-0.53493154,  0.00095151, -0.16959822, -0.0155062 , -0.07255006,
        -0.38704506, -0.32898626, -0.1718798 ,  0.36542615,  0.15575126]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.06369345, 0.10884935, 0.09178189, 0.10707261, 0.1011357 ,
        0.073845  , 0.07825925, 0.09157272, 0.15671666, 0.12707336]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [11]:
loss_fn(y_train[:1], predictions).numpy()

2.605787

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2925 - accuracy: 0.9148
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1428 - accuracy: 0.9582
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1078 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0890 - accuracy: 0.9722
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0770 - accuracy: 0.9754


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [14]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0732 - accuracy: 0.9782


[0.07315120100975037, 0.9782000184059143]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[6.4847121e-08, 5.0197571e-09, 2.3615001e-05, 5.9095892e-05,
        1.8101168e-11, 3.5811571e-08, 6.9754684e-12, 9.9991596e-01,
        1.9899063e-07, 1.0212480e-06],
       [5.0492556e-08, 4.3078606e-05, 9.9994600e-01, 1.0727439e-05,
        1.1865500e-16, 1.5131720e-07, 4.0346116e-08, 1.4061225e-13,
        8.1392759e-09, 1.1427292e-13],
       [1.3503156e-07, 9.9913567e-01, 3.4881057e-05, 1.5173236e-05,
        8.8414818e-05, 2.6842659e-07, 3.1993517e-05, 6.2642718e-04,
        6.2036968e-05, 5.0723793e-06],
       [9.9955350e-01, 1.9880463e-08, 4.1442367e-05, 4.6107337e-07,
        1.6740470e-06, 3.7374522e-07, 3.9194213e-04, 5.8124915e-06,
        4.1421167e-08, 4.7698441e-06],
       [1.4871321e-07, 4.2746201e-08, 1.6214079e-06, 3.0891858e-09,
        9.9821401e-01, 6.6921482e-09, 6.7614519e-07, 9.3473736e-06,
        1.4590536e-06, 1.7728154e-03]], dtype=float32)>